In [5]:
import pandas as pd
import psycopg2

In [6]:
# Configurações de conexão ao Redshift
host = 'aeq-bi.czukcqgshfeu.us-east-1.redshift.amazonaws.com'
dbname = 'bi'
user = 'temp'
password = 'vMrrNWTaG$7N'
port = '5439'

# Construa a string de conexão
conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"

In [7]:
# Função para obter os dados do Redshift
def get_data_from_redshift():
    try:
        # Conecte-se ao Redshift
        conn = psycopg2.connect(conn_string)

        # Sua consulta SQL para obter dados com origem e destino
        query = "SELECT * FROM p688.origndestinationseed limit 1000"

        # Execute a consulta
        df = pd.read_sql_query(query, conn)

        # Feche a conexão
        conn.close()

        return df

    except Exception as e:
        print("Ocorreu um erro ao obter dados do Redshift:", e)
        return None

# Carregue os dados do Redshift
df = get_data_from_redshift()

/var/folders/kc/z6ltzndn4vg0yhr3dj6lt_x40000gq/T/ipykernel_20816/1797433619.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [8]:
import pandas as pd
import folium
from folium import PolyLine, Marker
import ipywidgets as widgets
from IPython.display import display

# Crie uma função para atualizar as opções do filtro de rota com base na seleção de zona de tráfego
def atualizar_rotas(change):
    if change.new:
        filtro_route.options = df[df['trafficzoneidorign'].isin(change.new)]['route'].unique()

# Crie uma função para plotar o mapa com base nas seleções feitas
def plotar_mapa(zonas_selecionadas, ticketdate_selecionada, route_selecionada):
    # Filtra o DataFrame com base nas seleções
    df_filtrado = df[df['trafficzoneidorign'].isin(zonas_selecionadas) &
                     (df['ticketdate'] == ticketdate_selecionada) &
                     (df['route'] == route_selecionada)]

    # Verifique se há pelo menos uma linha válida no DataFrame filtrado
    if not df_filtrado.empty:
        # Crie um mapa centralizado nas coordenadas médias das origens
        mapa = folium.Map(location=[df_filtrado['latorigem'].mean(), df_filtrado['lonorigem'].mean()], zoom_start=10)

        for index, row in df_filtrado.iterrows():
            origem = (row['latorigem'], row['lonorigem'])
            destino = (row['latdestino'], row['londestino'])
            rota = row['route']
            zona = row['trafficzoneidorign']
            
            # Obtenha a contagem de usercards para esta rota
            usercards_count = df_filtrado[df_filtrado['route'] == rota]['usercard'].count()
            
            # Crie um pop-up com informações sobre a rota e o número de usercards
            popup_text = f"Origem: {origem}<br>Destino: {destino}<br>Rota: {rota}<br>Usercards: {usercards_count}"
            
            # Defina a cor do marcador da origem para ser diferente da cor do destino
            cor = 'green' if origem == (row['latorigem'], row['lonorigem']) else 'red'
            
            # Adicione marcadores de origem e destino com a cor apropriada
            folium.Marker(location=origem, icon=folium.Icon(color=cor), tooltip=popup_text).add_to(mapa)
            folium.Marker(location=destino, icon=folium.Icon(color='blue'), tooltip=popup_text).add_to(mapa)

            linha = PolyLine(locations=[origem, destino], color='blue', tooltip=popup_text)
            linha.add_to(mapa)

        display(mapa)
    else:
        print("Nenhum dado encontrado para as seleções feitas.")

# Crie um widget de seleção múltipla para a zona de tráfego de origem
zonas_de_trafego = df['trafficzoneidorign'].unique()
filtro_zona_trafego = widgets.SelectMultiple(
    options=zonas_de_trafego,
    description='Zona de Tráfego:',
    disabled=False,
)

# Crie widgets de seleção para o ticketdate e a rota
ticketdates = df['ticketdate'].unique()
filtro_ticketdate = widgets.Dropdown(
    options=ticketdates,
    description='Ticket Date:',
    disabled=False,
)
filtro_route = widgets.Dropdown(
    options=df['route'].unique(),
    description='Route:',
    disabled=False,
)

# Defina um observador para atualizar as opções do filtro de rota com base na seleção de zona de tráfego
filtro_zona_trafego.observe(atualizar_rotas, 'value')

# Exiba os widgets de seleção
widgets.interactive(plotar_mapa, zonas_selecionadas=filtro_zona_trafego, 
                    ticketdate_selecionada=filtro_ticketdate, route_selecionada=filtro_route)


interactive(children=(SelectMultiple(description='Zona de Tráfego:', options=(1, 92, 3, 61, 76, 4, 97, 77, 34,…